# Change Constraints

In a stochastic program we can put uncertainity also on the constraints.
This is called change constraints. we will look at the following types of change constraints:

- single chance
- joint chance constraints
- individual chance constraints

## mathematical expression

Mathematically we can express this as follows:

$$
\begin{array}{cc}
\min_x & c\cdot x \\
s.t.   & P(Ax\leq b) \geq p\\
       & x\geq 0
\end{array}
$$

Here the symbols are denoting the following
- $x\in\mathbb{R}^n$ the decision variable, 
- $c\in\mathbb{R}^n$ the coefficients in the objective function,
- $A\in\mathbb{R}^{m\times n}$ coefficient of the constraints,
- $b\in\mathbb{R}^m$ the right hand side of the constraints and
- $p\in(0,1]$ is the probability that the constraints (or some of them) may be violated.

## mathemathical solution

we can convert a stochastic program with change constraints into a mixed-integer problem as follows:
Let $S$ denote the set of scenarios and $y_k$ binary variables for each $k\in S$, i.e. $y_k$ equals 1 if and only if the constraint is satisfied in the corresponding scenario, then a scenario-based formulation of the above chance constrained stochastic program is given by:

$$
\begin{array}{cc}
\min & c x \\
s.t. & A_kx \leq b_k + M_k(1-y_k) \\
     & \sum_{k\in S} y_k \geq p\cdot |S| \\
     & x\geq0\\
     & y\in \{0,1\}^{|S|}
\end{array}
$$

where $M_k\in\mathbb{R}^m$ is a suitable choosen big-M vector.

## Pyomo examples with single change constraints

In [ ]:
import pyomo.environ as pyo

In [ ]:
I = [1,2]
S = [1,2,3,4]
Omega = {1:1, 2:2, 3:3, 4:4}
prob = {1:0.25, 2:0.25, 3:0.25, 4:0.25}
p = 0.75

In [ ]:
def cc(p, I, S, prob, omega):
    ## sets
    m = pyo.ConcreteModel()
    m.I = pyo.Set(initialize = I)
    m.S = pyo.Set(initialize = S)
    
    ## parameter
    @m.Param(m.S)
    def omega(m,s):
        return Omega[s]
    
    @m.Param(m.S)
    def py(m,s):
        return prob[s]
    
    m.M = pyo.Param(initialize = 10)
    m.p = pyo.Param(initialize = 1-p, doc = 'risk tolerance')
    
    # vars
    m.x = pyo.Var(m.I, domain = pyo.NonNegativeReals)
    m.y = pyo.Var(m.S, domain = pyo.Binary)
    m.cc = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0, m.p))
    
    # objective
    m.OBJ = pyo.Objective(expr = sum(m.x[i] for i in m.I), sense = pyo.minimize)
    
    # constraints
    @m.Constraint(m.S)
    def c1(m, s):
        return m.omega[s] * m.x[1] + m.x[2] >= 7 + m.M * (1 - m.y[s])
    
    m.cc_def = pyo.Constraint(expr = m.cc == 1 - sum(m.py[k] * m.y[k] for k in m.S))
    
    solver = pyo.SolverFactory('glpk')
    solver.solve(m)
    
    return m

In [ ]:
m = cc(p, I,S,prob,Omega)
m.pprint()

2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}

4 Param Declarations
    M : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :    10
    omega : Size=4, Index=S, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     1
          2 :     2
          3 :     3
          4 :     4
    p : risk tolerance
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :  0.25
    py : Size=4, Index=S, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  0.25
          2 :  0.25
          3 :  0.25
          4 :  0.25

3 Var Declarations
    cc : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : St